In [ ]:
%matplotlib notebook
import numpy as np
from matplotlib.pyplot import plot as pl
from matplotlib import cm

from scipy import signal

import ipyvolume as ipv
exec(open('python_functions/transfer_function_explorer.py').read())

# Pen-and paper circuit design

- ## Rules and tricks to simplify calculations
- ## Designing with individual transistors
- ## Operational amplifiers
- ## Exploring the Laplace transformation


## Rules and tricks
- Inspect circuit to find the first relevant quantities.
- Use the principle of superposition of effects
    - Choose a zero potential reference
    - Create auxiliary circuits to calculate individual contributions.
- Use Ohm's law
- Use operational calculus (Oliver Heaviside) for differential equations
$$i=C\frac{dv}{dt}\leftrightarrow I\cdot~e^{j\omega~t+j\phi}=C\cdot\frac{d}{dt}V\cdot~e^{j\omega~t+j\phi}\leftrightarrow~I\cdot~e^{j\omega~t+j\phi}=j\omega~C\cdot~V \cdot~e^{j\omega~t+j\phi}$$

$$\frac{v}{i}=Z=\frac{1}{j\omega~C}=\frac{1}{sC} $$

$$v=L\frac{di}{dt}\leftrightarrow V\cdot~e^{j\omega~t+j\phi}=\cdot\frac{d}{dt}I\cdot~e^{j\omega~t+j\phi}\leftrightarrow~V\cdot~e^{j\omega~t+j\phi}=j\omega~L\cdot~I\cdot~e^{j\omega~t+j\phi}$$

$$\frac{v}{i}=Z={j\omega~L}=sL $$

# Beyond pen and paper..

## Spice simulator

- ### My suggestion is [LTSpice from Analog Devices](https://www.analog.com/en/design-center/design-tools-and-calculators/ltspice-simulator.html)

- ### There is an unofficial wiki [ltwiki.org](https://ltwiki.org) that allows to unlock undocumented capabilities

# The bipolar transistor

[2n2222 NPN](https://www.onsemi.com/pdf/datasheet/p2n2222a-d.pdf)

[2n2907 PNP](https://www.onsemi.com/pdf/datasheet/p2n2907a-d.pdf)

## Small signal model
beta $$ \beta=\frac{I_c}{I_b} $$
transconductance $$ g_m = \frac{q\cdot~I_c}{kT}=\frac{I_c}{V_t} $$
Imput resistance $$ R_\pi = \frac {\beta}{V_t} $$


# The three gain configurations

# Some basic element of control theory: feedback
- [Harold S. Black patented feedback amplifiers in 1928-1937](https://patentimages.storage.googleapis.com/b8/63/f0/e6bfd23228c3e1/US2102671.pdf)

<img src="./images/Black.svg" > </img>


# A simple Op-amp

- A good resource about Opamps is [Linear Circuit Design Handbook by Analog Devices](https://www.analog.com/en/education/education-library/linear-circuit-design-handbook.html)

Op-amps have been around since...\*
- First time the term was used: [John R. Ragazzini, et al., Proceedings of the IRE, vol. 35, May 1947, pp. 444-452.](https://worldradiohistory.com/Archive-IRE/40s/IRE-1947-05.pdf)
    
- [George A. Philbrick Researches (GAP/R) K2-W](http://www.philbrickarchive.org/k2-w_refurbished.pdf) commercialized in 1953

- Transistor hybrid opamps
- First monolitic opamp μA702 (Bob Widlar) in 1963 \*.



\* [*opamp history by Analog Devices*](https://www.analog.com/media/en/training-seminars/design-handbooks/Op-Amp-Applications/SectionH.pdf)






# Let's open an Op-amp datasheet

[op27](https://www.ti.com/lit/ds/symlink/op27.pdf): general purpose low noise opamp

[opa847](https://www.ti.com/lit/ds/symlink/opa847.pdf): high speed opamp not unitary gain stable

# Laplace transform

[Laplace transform from Wolfram Mathworld](https://mathworld.wolfram.com/LaplaceTransform.html)

Fourier transform $$ \mathcal{F}\left( f(x)\right)=\int_{-\infty}^{\infty} f(t) e^{-i 2\pi f t} dt$$

Laplace transform $$ \mathcal{L}\left( f(x)\right)=\int_{0}^{\infty} f(x) e^{-st} dt$$
Complex frequency paramater: $$ s = \sigma + i\omega $$

Antitransform is = Fourier transform only if all poles and zeros are on left plane!


# Laplace transform notation

Zero-pole-gain form : $$ \frac{k\cdot(\frac{1}{\tau_a}+s)...}{(\frac{1}{\tau_b}+s)...} $$

Canonical form: $$ \frac{k\cdot\tau_b\cdot(1+\tau_as)...}{\tau_a\cdot(1+\tau_bs)...}$$



In [ ]:
ipv.figure(); ipv.show()

zeros = [-4]
poles = [0]
#poles = [-0.25+2j,-0.25-2j]
gain = 1

system = signal.ZerosPolesGain(zeros,poles,gain)


laplace_plot(system, surface=True) #This is a custom function: not from a toolset.



# Useful tools for stability analisys

- Python library [control](https://python-control.readthedocs.io/en/0.9.2/control.html)
    - Bode plot
    - Poles-zeros map
    - Root locus (Available as online tool at Swarthmore College)(https://lpsa.swarthmore.edu/Root_Locus/RLDraw.html)
    
    drawing the roots of $ 1+k\cdot~H(s)G(s) $ as k increases
    ![control](images/control_system.svg)

In [ ]:
import control as c #Control is a useful control system toolbox
import scipy.signal as sy
from matplotlib import pyplot as plt
%matplotlib notebook

zeros = [-10]
poles = [0,-20,-50]

#zeros = [-0.05]
zeros = []
#poles = [-0.1+0.4j,-0.1-0.4j]

gain = np.product(np.abs([p for p in poles if p != 0]+[1]))/np.product(np.abs(zeros+[1]))


F = sy.ZerosPolesGain(zeros,poles,gain*300)


system = F.to_tf()

sys = c.tf(system.num,system.den)

plt.figure()



#c.pzmap(sys)
#c.bode(sys,Hz=True)

c.rlocus(sys)


plt.figure()
imp = c.impulse_response(sys)
plt.plot(imp.time,imp.outputs)


plt.figure()

n=sys/(sys+1)

#c.bode(n);
imp = c.impulse_response(n)
plt.plot(imp.time,imp.outputs)








![PID controller](./images/PID_Schematic.png)


In [ ]:
#NOTE: The code below uses just scipy library "signal" instead of control
## Hidden cell

import scipy.signal as sy
from matplotlib import pyplot as plt
%matplotlib notebook


zeros = [-1]

poles = [0,-10]

gain = 3

F = sy.ZerosPolesGain(zeros,poles,gain)

w,mag,phase = sy.bode(F)

plt.semilogx(w,mag)
plt.semilogx(w,phase)


